In [1]:
import os, sys
sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))

from src.utils.utils import *

# Dicionário de metadados

## Metadados cabeçalho

Cabeçalho Dicom - CBIS-DDSM

In [2]:

dictionary_metadata_cbis_ddsm = {}
dictionary_empty_meta = {}

metadata_cbis_ddsm = load_json("extracted_metadata_CBIS-DDSM",  "../../outputs/mamografia/extracted_metadata")

for current_meta_study in metadata_cbis_ddsm: # itera sobre os estudos
    tmp_meta = current_meta_study['metadata_dicom_files']
    
    all_tags = set()
    empty_tags = set()
    
    meta_dicom_files = tmp_meta['original'] + tmp_meta['cropped']
    
    for meta_dicom_file in meta_dicom_files: # Itera sobre os arquivos dicom no estudo
        for key, value in meta_dicom_file.items(): # Iterar sobre as tags no arquivo dicom
            all_tags = all_tags | set(meta_dicom_file.keys()) # união
            empty_tags = empty_tags | set([key for key, value in meta_dicom_file.items() 
                                            if meta_dicom_file[key] == ""]) # união
    update_count_tag(all_tags, dictionary_metadata_cbis_ddsm)
    update_count_tag(empty_tags, dictionary_empty_meta) 
        
for key in dictionary_metadata_cbis_ddsm.keys():
    if key in dictionary_empty_meta.keys():
        dictionary_metadata_cbis_ddsm[key] -= dictionary_empty_meta[key]
        
save_json("dictionary_dicom_CBIS-DDSM", dictionary_metadata_cbis_ddsm, f"../../outputs/mamografia/dictionary_metadata/dicom_pgm")

Cabeçalho Dicom - CMMD

In [3]:
metadata_cmmd = load_json("extracted_metadata_CMMD",  "../../outputs/mamografia/extracted_metadata")

dictionary_metadata_cbis_ddsm = {}
dictionary_empty_meta = {}

for current_meta in metadata_cmmd: # Iterar sobre os estudos
    meta_dicom_files = current_meta['metadata_dicom_files']
    
    all_tags = set()
    empty_tags = set()
    
    for current_meta_dicom in meta_dicom_files: # Iterar sobre os arquivos dicom no estudo
        for key, value in current_meta_dicom.items(): # Iterar sobre as tags no arquivo dicom
            all_tags = all_tags | set(current_meta_dicom.keys()) # união
            empty_tags = empty_tags | set([key for key, value in current_meta_dicom.items() 
                                           if current_meta_dicom[key] == ""]) # união
    
            if isinstance(value, list):
                for seq_item in value:
                    all_tags = all_tags | set(seq_item.keys()) # união
                    empty_tags = empty_tags | set([key for key, value in seq_item.items() 
                                           if seq_item[key] == ""]) # união
                    
    update_count_tag(all_tags, dictionary_metadata_cbis_ddsm)
    update_count_tag(empty_tags, dictionary_empty_meta) 
    
for key in dictionary_metadata_cbis_ddsm.keys():
    if key in dictionary_empty_meta.keys():
        dictionary_metadata_cbis_ddsm[key] -= dictionary_empty_meta[key]

save_json("dictionary_dicom_CMMD", dictionary_metadata_cbis_ddsm, f"../../outputs/mamografia/dictionary_metadata/dicom_pgm") 

Cabelçalho Dicom - INBREAST

In [4]:
metadata_inbreast = load_json("extracted_metadata_INBREAST",  "../../outputs/mamografia/extracted_metadata")

dictionary_metadata_cbis_ddsm = {}
dictionary_empty_meta = {}

for current_meta in metadata_inbreast: # Iterar sobre os estudos
    meta_dicom_file = current_meta['metadata_dicom_files']
    
    all_tags = set()
    empty_tags = set()
    
    for key, value in meta_dicom_file.items(): # Iterar sobre as tags no arquivo dicom
        all_tags = all_tags | set(meta_dicom_file.keys()) # união
        empty_tags = empty_tags | set([key for key, value in meta_dicom_file.items() 
                                       if meta_dicom_file[key] == ""]) # união
            
    update_count_tag(all_tags, dictionary_metadata_cbis_ddsm)
    update_count_tag(empty_tags, dictionary_empty_meta) 
    
for key in dictionary_metadata_cbis_ddsm.keys():
    if key in dictionary_empty_meta.keys():
        dictionary_metadata_cbis_ddsm[key] -= dictionary_empty_meta[key]

save_json("dictionary_dicom_INBREAST", dictionary_metadata_cbis_ddsm, f"../../outputs/mamografia/dictionary_metadata/dicom_pgm")

Cabeçalho PGM - MIAS

In [5]:
metadata_mias = load_json("extracted_metadata_MIAS",  "../../outputs/mamografia/extracted_metadata")

dictionary_metadata_cbis_ddsm = {}

for current_meta in metadata_mias:
    metadata_pgm_file = current_meta['metadata_pgm_file']
    """ normalized_dict = {}
    
    #Criar dicionário com tags normalizada
    normalized_dict["rows (0028, 0010)"] = metadata_pgm_file['rows']
    normalized_dict["columns (0028, 0011)"] = metadata_pgm_file['columns']
    
    bits_used = get_bits_allocated(metadata_pgm_file['max_value'])
    normalized_dict["bits_allocated (0028, 0100)"] = bits_used
    normalized_dict["bits_stored (0028, 0101)"] = bits_used
    normalized_dict["high_bit (0028, 0102)"] = bits_used - 1"""
    
    for key, value in metadata_pgm_file.items():
        if value == "" or key == "img_path":
            del metadata_pgm_file[key]
    
    update_count_tag(metadata_pgm_file.keys(), dictionary_metadata_cbis_ddsm)
    
save_json("dictionary_pgm_MIAS", dictionary_metadata_cbis_ddsm, f"../../outputs/mamografia/dictionary_metadata/dicom_pgm")

Juntar Dicionários de tags do cabelhaço Dicom - CMMD, DDSM e INBREAST

In [6]:
dictionary_cmmd = load_json("dictionary_dicom_CMMD", "../../outputs/mamografia/dictionary_metadata/dicom_pgm")
dictionary_cbis_ddsm = load_json("dictionary_dicom_CBIS-DDSM", "../../outputs/mamografia/dictionary_metadata/dicom_pgm")
dictionary_inbreast = load_json("dictionary_dicom_INBREAST", "../../outputs/mamografia/dictionary_metadata/dicom_pgm")

full_dictionary = merge_dictionary(dictionary_cmmd, dictionary_cbis_ddsm)
full_dictionary = merge_dictionary(full_dictionary, dictionary_inbreast)

save_json("full_dictionary_dicom_pgm", full_dictionary, "../../outputs/mamografia/dictionary_metadata/dicom_pgm")

## Metadados arquivos CSV e TXT

In [10]:
# CMMD
metadata_cmmd = load_json("extracted_metadata_CMMD", "../../outputs/mamografia/extracted_metadata")               
dictionary_metadata_cmmd = create_dict_meta(metadata_cmmd, 'csv')
save_json("dictionary_csv_CMMD", dictionary_metadata_cmmd, "../../outputs/mamografia/dictionary_metadata/csv_txt")

# CBIS - DDSM
metadata_cbis_ddsm = load_json("extracted_metadata_CBIS-DDSM", "../../outputs/mamografia/extracted_metadata")
dictionary_metadata_cbis_ddsm = create_dict_meta(metadata_cbis_ddsm, 'csv')
save_json("dictionary_csv_CBIS-DDSM", dictionary_metadata_cbis_ddsm, "../../outputs/mamografia/dictionary_metadata/csv_txt")

# INBREAST
metadata_inbreast = load_json("extracted_metadata_INBREAST", "../../outputs/mamografia/extracted_metadata")
dictionary_metadata_inbreast = create_dict_meta(metadata_inbreast, 'csv')
save_json("dictionary_csv_INBREAST", dictionary_metadata_inbreast, "../../outputs/mamografia/dictionary_metadata/csv_txt")

# MIAS
metadata_mias = load_json("extracted_metadata_MIAS", "../../outputs/mamografia/extracted_metadata")           
dictionary_metadata_mias = create_dict_meta(metadata_mias, 'txt')
save_json("dictionary_txt_MIAS", dictionary_metadata_mias, "../../outputs/mamografia/dictionary_metadata/csv_txt")

# Juntar e salvar os dicionários de metadados
full_dictionary = merge_dictionary(dictionary_metadata_cmmd, dictionary_metadata_cbis_ddsm)
full_dictionary = merge_dictionary(full_dictionary, dictionary_metadata_inbreast)
save_json("full_dictionary_metadata", full_dictionary, "../../outputs/mamografia/dictionary_metadata/csv_txt")